# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
# Create a keyspace if it doesn't exist

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity_project
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Using Keyspace udacity_project

try:
    session.execute("USE udacity_project")
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
# Reading and displaying the first 5 rows of the CSV file

df = pd.read_csv('event_datafile_new.csv')

df.head(5)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Des'ree,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",139,You Gotta Be,8
1,Mr Oizo,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",139,Flat 55,8
2,Tamba Trio,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",139,Quem Quiser Encontrar O Amor,8
3,The Mars Volta,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",139,Eriatarka,8
4,Infected Mushroom,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",139,Becoming Insane,8


In [9]:
# For table music_sessions, the sessionId was chosen as the partition key and itemInSession as clustering columns.
# Explanation: The PRIMARY KEY (sessionId, itemInSession) enables filtering and retrieval of specific sessions and their items efficiently.
# The query selects artist, song, and length for a given session (sessionId = 338) and item in session (itemInSession = 4) from the music_sessions table.

try:
    query = "CREATE TABLE IF NOT EXISTS music_sessions "
    query += "(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY (sessionId, itemInSession))"
    session.execute(query)
except Exception as e:
    print(e)

In [10]:
nomes_colunas = df.columns

# Iterating to print the name of each column
for indice, nome_coluna in enumerate(nomes_colunas):
    print(f"Nome da coluna: {nome_coluna}, line[{indice}]")

Nome da coluna: artist, line[0]
Nome da coluna: firstName, line[1]
Nome da coluna: gender, line[2]
Nome da coluna: itemInSession, line[3]
Nome da coluna: lastName, line[4]
Nome da coluna: length, line[5]
Nome da coluna: level, line[6]
Nome da coluna: location, line[7]
Nome da coluna: sessionId, line[8]
Nome da coluna: song, line[9]
Nome da coluna: userId, line[10]


In [11]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_sessions (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [12]:
try:
    query = "SELECT artist, song, length \
             FROM music_sessions \
             WHERE sessionid = 338 AND itemInSession = 4"
    rows = session.execute(query)

    for row in rows:
        print(
            f"{row.artist.ljust(30)}{row.song.ljust(40)}{str(row.length).ljust(20)}"
        )
except Exception as e:
    print(f"Ocorreu um erro: {e}")

Faithless                     Music Matters (Mark Knight Dub)         495.30731201171875  


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [13]:
nomes_colunas = df.columns

# Iterating to print the name of each column
for indice, nome_coluna in enumerate(nomes_colunas):
    print(f"Nome da coluna: {nome_coluna}, line[{indice}]")

Nome da coluna: artist, line[0]
Nome da coluna: firstName, line[1]
Nome da coluna: gender, line[2]
Nome da coluna: itemInSession, line[3]
Nome da coluna: lastName, line[4]
Nome da coluna: length, line[5]
Nome da coluna: level, line[6]
Nome da coluna: location, line[7]
Nome da coluna: sessionId, line[8]
Nome da coluna: song, line[9]
Nome da coluna: userId, line[10]


In [14]:
# For table music_user_sessions, the composite partition key consists of userid and sessionid, and iteminsession serves as a clustering column.
# Explanation: The PRIMARY KEY ((userid, sessionid), iteminsession) efficiently organizes data by user sessions and items within sessions.
# The output includes the sorted song list by itemInSession and displays the user's firstname and lastname alongside the corresponding artist and song.


try:
    query = "CREATE TABLE IF NOT EXISTS music_user_sessions "
    query += "(userid int, sessionid int, iteminsession int, artist text, song text, firstname text, lastname text, PRIMARY KEY ((userid, sessionid), iteminsession))"
    session.execute(query)
except Exception as e:
    print(e)

with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO music_user_sessions (userid, sessionid, iteminsession, artist, song, firstname, lastname)"
        query += " VALUES (%s, %s, %s, %s, %s, %s, %s)" 
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [15]:
try:
    query = "SELECT artist, song, firstname, lastname \
             FROM music_user_sessions \
             WHERE userid = 10 AND sessionid = 182"
    rows = session.execute(query)

    for row in rows:
        print(
            f"{row.artist[:19].ljust(20)}{row.song[:39].ljust(40)}{row.firstname} {row.lastname}"
        )

except Exception as e:
    print(f"Ocorreu um erro: {e}")

Down To The Bone    Keep On Keepin' On                      Sylvie Cruz
Three Drives        Greece 2000                             Sylvie Cruz
Sebastien Tellier   Kilometer                               Sylvie Cruz
Lonnie Gordon       Catch You Baby (Steve Pitron & Max Sann Sylvie Cruz


In [16]:
# For table music_history_by_song, the song column serves as the partition key and userid as additional clustering column.
# Explanation: The PRIMARY KEY (song, userid) efficiently stores user-specific listening history for each song.
# The output displays the first and last names of all users who have listened to this specific song in the music app history.


try:
    query = "CREATE TABLE IF NOT EXISTS music_history_by_song "
    query += "(song text, userid int, firstname text, lastname text, PRIMARY KEY (song, userid))"
    session.execute(query)
except Exception as e:
    print(e)

with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO music_history_by_song (song, userid, firstname, lastname)"
        query += " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [17]:
try:
    query = "SELECT firstname, lastname \
             FROM music_history_by_song \
             WHERE song = 'All Hands Against His Own'"
    rows = session.execute(query)

    for row in rows:
        print(
            f"{row.firstname} {row.lastname}"
        )

except Exception as e:
    print(f"Ocorreu um erro: {e}")

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [18]:
try:
    session.execute("DROP TABLE IF EXISTS music_history_by_song")
    session.execute("DROP TABLE IF EXISTS music_user_sessions")
    session.execute("DROP TABLE IF EXISTS music_sessions")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()